# Cobalt Strike Notebook

## Stager Payload Analysis

This notebook can be used to emulate the shellcode provided as a stager by Cobalt Strike. 

This can be useful to provide information on the HTTP hostname, port, path and cookie used by CS to request the `beacon.bin` payload.

In [ ]:
import sys
sys.path.append('../src/')

from helpers import emulation
import ipywidgets as widgets
import pandas as pd

SHELLCODE_ARCH = 'x86'

def hook_internetconnect(emu, api_name, func, params):
    rv = func(params)
    print(f"Host: {params[1]}:{params[2]}")
    return rv

def hook_httpopenrequest(emu, api_name, func, params):
    rv = func(params)
    print(f"Path: {params[2]}")
    return rv

def hook_httpsendrequest(emu, api_name, func, params):
    rv = func(params)
    print(f"Cookie: {params[1]}")
    emu.stop()
    return rv

def on_upload_change(change):
    print("File uploaded... emulating.")
    for f in uploader.value:
        emu = emulation.ShellcodeEmulation.usingData(uploader.value[f]['content'], SHELLCODE_ARCH)
        emu.addHook('wininet','HttpOpenRequestA', hook_httpopenrequest)
        emu.addHook('wininet','HttpSendRequestA', hook_httpsendrequest)
        emu.addHook('wininet','InternetConnectA', hook_internetconnect)
        emu.run()

uploader = widgets.FileUpload(multiple=False)
uploader.observe(on_upload_change, names='_counter')
display(uploader)